In [2]:
import pandas as pd
import tqdm
import numpy as np

In [3]:
df = pd.read_csv('Data/gun-violence-data_01-2013_03-2018.csv')
df.describe()

,incident_id,n_killed,n_injured,congressional_district,latitude,longitude,n_guns_involved,state_house_district,state_senate_district
count,2.396770e+05,239677.000000,239677.000000,227733.000000,231754.000000,231754.000000,140226.000000,200905.000000,207342.00000
mean,5.593343e+05,0.252290,0.494007,8.001265,37.546598,-89.338348,1.372442,55.447132,20.47711
std,2.931287e+05,0.521779,0.729952,8.480835,5.130763,14.359546,4.678202,42.048117,14.20456
min,9.211400e+04,0.000000,0.000000,0.000000,19.111400,-171.429000,1.000000,1.000000,1.00000
25%,3.085450e+05,0.000000,0.000000,2.000000,33.903400,-94.158725,1.000000,21.000000,9.00000
50%,5.435870e+05,0.000000,0.000000,5.000000,38.570600,-86.249600,1.000000,47.000000,19.00000
75%,8.172280e+05,0.000000,1.000000,10.000000,41.437375,-80.048625,1.000000,84.000000,30.00000
max,1.083472e+06,50.000000,53.000000,53.000000,71.336800,97.433100,400.000000,901.000000,94.00000


In [5]:
df.describe(include=['O'])

,date,state,city_or_county,address,incident_url,source_url,gun_stolen,gun_type,incident_characteristics,location_description,notes,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources
count,239677,239677,239677,223180,239677,239209,140179,140226,239351,42089,158660,147379,197558,203315,117424,15774,212051,214814,239068
unique,1725,51,12898,198037,239677,213989,349,2502,18126,27595,136652,18951,898,873,113488,284,2150,259,217280
top,2017-01-01,Illinois,Chicago,2375 International Pkwy,http://www.gunviolencearchive.org/incident/296451,http://blog.tsa.gov,0::Unknown,0::Unknown,Shot - Wounded/Injured,Austin,man shot,0::24,0::Adult 18+,0::Male,0::Officer,1::Significant others - current or former,0::Injured,0::Victim,http://blog.tsa.gov
freq,342,17556,10814,160,1,1092,121310,93559,47541,240,501,3814,94671,93496,144,2651,42293,58564,1092


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 239677 entries, 0 to 239676
Data columns (total 29 columns):
incident_id                    239677 non-null int64
date                           239677 non-null object
state                          239677 non-null object
city_or_county                 239677 non-null object
address                        223180 non-null object
n_killed                       239677 non-null int64
n_injured                      239677 non-null int64
incident_url                   239677 non-null object
source_url                     239209 non-null object
incident_url_fields_missing    239677 non-null bool
congressional_district         227733 non-null float64
gun_stolen                     140179 non-null object
gun_type                       140226 non-null object
incident_characteristics       239351 non-null object
latitude                       231754 non-null float64
location_description           42089 non-null object
longitude                    

In [1]:
#for f in df.columns:
#    print(f + " |  | " + str(df.iloc[0][str(f)]))

#print(df.head())
#print(len(df))

#H1 In looking at the table columns we can see that there is a lot of information here for us to digest

Column Name | Description | Example
--- | --- | ---
incident_id | Unique Id of data point | 461105
date | Date | 2013-01-01
state | State of occurance | Pennsylvania
city_or_county | city or county | Mckeesport
address | address if exists | 1506 Versailles Avenue and Coursin Street
n_killed | Number of people killed | 0
n_injured | Number of people injured | 4
incident_url | Archive citation | http://www.gunviolencearchive.org/incident/461105
source_url | Actual Citation | http://www.post-gazette.com/local/south/2013/01/17/Man-arrested-in-New-Year-s-Eve-shooting-in-McKeesport/stories/201301170275
incident_url_fields_missing | If urls do not exist | False
congressional_district | district | 14.0
gun_stolen | Stolen Gun | nan
gun_type | Type of gun | nan
incident_characteristics | List of possible | 'Shot - Wounded/Injured'
latitude | lat | 40.3467
location_description | description of location | nan
longitude | lon | -79.8559
n_guns_involved | Number of guns involved | nan
notes | Notes | Julian Sims under investigation: Four Shot and Injured
participant_age | List of participants and ages | 0::20
participant_age_group | List of participant ages | 0::Adult 18+
participant_gender | list of participants and gender | 0::Male
participant_name | Names | 0::Julian Sims
participant_relationship | Relationship to shooter | nan
participant_status | Status of involved | '0::Arrested'
participant_type | People status | 0::Victim
sources | Source URLs | 'http://pittsburgh.cbslocal.com/2013/01/01/4-people-shot-in-mckeesport/'
state_house_district | District | nan
state_senate_district | District| nan


Several of the Cell columns, including the participant_ columns and the list of incident_characteristics are in the form of a list separated by double and single 'bars' || and |.

One portion of the analysis will be to split out these lists into a feature vectorization if they fall into certain categories.  Upon analyzing the incident_characteristics we see that there are 109 distinct possibilities among the 240,000 entries, therefore we will be able to gain some insight into the data by separating this out.

We will also attempt to tally the total number of participants and how many were victims vs perpatrators of the crime